In [7]:
import pandas as pd
import numpy as np
import pickle
from math import sqrt
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression, LassoCV, RidgeCV, ElasticNet
from sklearn.tree import DecisionTreeRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [1]:
pip install streamlit

     |████████████████████████████████| 8.2 MB 4.2 MB/s eta 0:00:01
     |████████████████████████████████| 74 kB 2.0 MB/s eta 0:00:011
     |████████████████████████████████| 4.2 MB 5.8 MB/s eta 0:00:01
Note: you may need to restart the kernel to use updated packages.


In [4]:
import streamlit as st

@st.cache
def prepared_data(df):
  students_df = df[list(('school', 'sex', 'age', 'Pstatus', 'studytime', \
                                  'failures', 'schoolsup', 'famsup', 'paid', 'activities', 'higher', \
                                  'internet', 'absences', 'G1', 'G2', 'G3'))]
  X = students_df.drop('G3', axis = 1)
  y = students_df.G3

  # Разбиение данных на обучающую и тестовую выборки
  (X_train, X_test, y_train, y_test) = train_test_split(X, y, test_size = 0.2)
  
  scaler = StandardScaler()
  scaler.fit(X_train)

  X_train = scaler.transform(X_train)
  X_test = scaler.transform(X_test)
  return X_train, y_train, X_test, y_test

In [6]:
def trainee_elastic_net_model(X_train, y_train, parameters={
    'alpha': 1,
    'fit_intercept': True,
    'max_iter': 1,
    'positive': True
}):
    return ElasticNet(
        alpha=parameters['alpha'],
        fit_intercept=parameters['fit_intercept'],
        max_iter=parameters['max_iter'],
        positive=parameters['positive']
    ).fit(X_train, y_train)

In [8]:
def process_prediction(model, X_test, y_test):
    y_pred = model.prediction(X_test)
    
    st.write('R^2: {}'.format(r2_score(y_test, y_pred)))
    st.write('MSE: {}'.format(mean_squared_error(y_test, y_pred)))
    st.write('MAE: {}'.format(mean_absolute_error(y_test, y_pred)))
    st.write('RMSE: {}'.format(mean_squared_error(y_test, y_pred, squared=False)))

In [9]:
def serialize_model(model):
    return pickle.dumps(model)

In [10]:
def deserialize_model(model):
    return pickle.loads(model)

In [11]:
def serialize_model_in_file(model, filename):
    with open(filename, 'wb') as file:
        pickle.dump(model)
    return filename

In [12]:
def deserialize_model_in_file(model, filename):
    with open(filename, 'rb') as file:
        model = pickle.load(model)
    return model

In [14]:
@st.cache
def load_data(filename='../data/students_preprocessed.csv'):
    return pd.read_csv(filename, sep = ',')

In [16]:
def main():
    page = st.sidebar('PAGE', ['Description', 'Regressor'])
    
    if page == 'Description':
        st.header('Students dataset description')
        st.write('''
Подготовить данные, убрать аномалии. Разбить датасет, провести обучение моделей: 
    Линейная регрессия;
    Регрессия дерева решений;
    LASSO;
    Гребневая регрессия;
    Elastic Net регрессия.

Найти наилучшие гиперпараметры.
Оценить качество моделей: R2, Mean Square Error(MSE), Root Mean Square Error(RMSE),  mean absolute error (MAE).
        ''')
        st.write('''
Источник: https://archive.ics.uci.edu/ml/datasets.php Описание: данные учащихся двух португальских школ, собраны путем проведения анкетирования учеников. Данные для регрессии и классификации. Суть задачи: собрать и обработать данные об успеваемости учеников по математике. Целевой признак: G3 - оценка третьего, финального курса математики (порядковый признак, упорядоченное множество чисел от 0 до 20). Объект: ученик. Признаки объектов:

school - школа (бинарный признак: "GP" - Габриэль Перейра или "MS" - Мусиньо да Сильвейра);
sex - пол студента (бинарный признак: 'F' - женщина или 'M' - мужчина);
age - возраст студента (порядковый признак, упорядоченное множество чисел от 15 до 22 лет);
address - тип домашнего адреса студента (бинарный признак: 'U' - городской или 'R' - сельский);
famsize - количество человек в семье (бинарный признак: 'LE3' - меньше или равно 3 или 'GT3' - больше 3);
Pstatus - статус совместного проживания родителей (бинарный признак: 'T' - совместное проживание или 'A' - порознь);
Medu - образование матери (порядковый признак, упорядоченное множество чисел от 0 до 4: 0 - Нет, 1 - начальное образование (4-й класс), 2 - 5-9-й класс, 3-среднее образование или 4-высшее образование);
Fedu - образование отца (порядковый признак, упорядоченное множество чисел от 0 до 4: 0 - Нет, 1 - начальное образование (4-й класс), 2 - 5-9-й класс, 3-среднее образование или 4-высшее образование);
Mjob - работа матери (категориальный признак: "teacher", "health", "services" (например, административные или полицейские), "at_home" или "other");
Fjob - работа отца (категориальный признак: "teacher", "health", "services" (например, административные или полицейские), "at_home" или "other");
reason - причина выбора данной школы (категориальный признак: "home" - школа расположена близко к дому, "reputation" - репутация школы, "course" - профильное обучения, "other" - другая причина);
guardian - опекун студента (категориальный признак: 'mather', 'father' или 'other');
traveltime - время, ежедневно затрачиваемое в пути в школу и обратно домой (порядковый признак, упорядоченное множество чисел от 1 до 4: 1 - меньше 15-ти минут, 2 - 15 - 30 минут, 3 - от 30 минут до одного часа, 4 - больше часа);
studytime - время, проведенное в школе за неделю (порядковый признак, упорядоченное множество чисел от 1 до 4: 1 - менее двух часов, 2 - от 2-х до 5-ти часов, 3 - от 5-ти до 10-ти часов, 4 - более 10-ти часов);
failures - количество неуспеваний на уроках (порядковый признак, упорядоченное множество чисел от 0 до 4);
schoolsup - занимается ли ученик дополнительным образованием (бинарный признак);
famsup - помогает ли семья в учебе (бинарный признак);
paid - берет ли ученик дополнительные платные занятия в рамках предмета курса (бинарный признак);
activities - участвует ли ученик во внеклассных школьных мероприятиях (бинарный признак);
nursery - посещал ли ученик детский сад (бинарный признак);
higher - планирует ли ученик получать высшее образование (бинарный признак);
internet - имеет ли ученик доступ в интернет (бинарный признак);
romantic - состоит ли ученик в романтических отношениях (бинарный признак);
famrel - семейная обстановка (порядковый признак, упорядоченное множество чисел от 1 до 5);
freetime - польза проведенного свободного времени для учебы (порядковый признак, упорядоченное множество чисел от 1 до 5);
goout - хорошо ли ученик ладит со своими одноклассниками (порядковый признак, упорядоченное множество чисел от 1 до 5);
Dalc - потребление алкоголя в будние дни (порядковый признак, упорядоченное множество чисел от 1 до 5);
Walc - потребление алкоголя в выходные дни (порядковый признак, упорядоченное множество чисел от 1 до 5);
health - состояние здоровья ученика (порядковый признак, упорядоченное множество чисел от 1 до 5);
absences - количество пропущенных уроков (порядковый признак, множество чисел от 0 до 93);
G1 - оценка первого курса математики (порядковый признак, упорядоченное множество чисел от 0 до 20);
G2 - оценка второго курса математики (порядковый признак, упорядоченное множество чисел от 0 до 20);
G3 - оценка третьего, финального курса математики (порядковый признак, упорядоченное множество чисел от 0 до 20).
        ''')
    else:
        st.header('Students dataset')
        df = load_data()
        
        alpha = st.slider('ALPHA',
                          min_value=0.1,
                          max_value=3.0,
                          value=0.1,
                          step=0.1,
                          key='alpha')
        fit_intercept = st.selectbox('FIT INTERCEPT',
                                     [True, False],
                                     'is_fit_intercept')
        max_iter = st.slider('MAX ITER',
                             min_value=0.1,
                             max_value=3.0,
                             value=0.1,
                             step=0.1,
                             key='max_iter')
        positive = st.selectbox('POSITIVE',
                                [True, False],
                                'is_positive')
        
        X_train, y_train, X_test, y_test = prepared_data(df)
        model = trainee_elastic_net_model(X_train, y_train, {
            'alpha': alpha,
            'fit_intercept': fit_intercept,
            'max_iter': max_iter,
            'positive': positive
        })
        #serialize_model = serialize_model(model)
        #deserialize_model = deserialize_model(model)
        process_prediction(model, X_test, y_test)

In [17]:
if __name__ == "__main__":
    main()

TypeError: 'DeltaGenerator' object is not callable